In [20]:
import os
import re
import numpy as np
from copy import copy
from tqdm import tqdm
from collections import Counter
from nltk import everygrams
from typing import Dict


np.random.seed(4)

DATA_PATH = "data"
RE_RU = re.compile("[^а-я ]*")
RE_EN = re.compile("[^a-z ]*")

SPACE = " "
ALPHABET_RU = "абвгдежзийклмнопрстуфхцчшщъыьэюя" + SPACE


TEST = """
Только окончится один месяц, сразу же начинается другой. И ни разу еще не бывало так, \
чтобы февраль пришел раньше, чем уйдет январь, а май обогнал бы апрель. \
Месяцы идут один за другим и никогда не встречаются. \
Но люди рассказывают, будто в горной стране Богемии была девочка, которая видела все двенадцать месяцев сразу. \
Как же это случилось? А вот как. \
В одной маленькой деревушке жила злая и скупая женщина с дочкой и падчерицей. \
Дочку она любила, а падчерица ничем ей не могла угодить. \
Что ни сделает падчерица — все не так, как ни повернется — все не в ту сторону. \ 
Дочка по целым дням на перине валялась да пряники ела, а падчерице с утра до ночи и присесть некогда было: \
то воды натаскай, то хворосту из лесу привези, то белье на речке выполощи, то грядки в огороде выполи. \
Знала она и зимний холод, и летний зной, и весенний ветер, и осенний дождь. \
Потому-то, может, и довелось ей однажды увидеть все двенадцать месяцев разом. \
Была зима. Шел январь месяц. Снегу намело столько, что от дверей его приходилось отгребать лопатами, \
а в лесу на горе деревья стояли по пояс в сугробах и даже качаться не могли, когда на них налетал ветер. \
Люди сидели в домах и топили печки.
"""

In [21]:
texts = []

with open(os.path.join(DATA_PATH, "AnnaKarenina.txt"), "r", encoding="utf-8") as file:
    text = file.read()
    texts.append(re.sub(RE_RU, "", text.lower()))
    
with open(os.path.join(DATA_PATH, "WarAndPeace.txt"), "r", encoding="utf-8") as file:
    text = file.read()
    texts.append(re.sub(RE_RU, "", text.lower()))
    
with open(os.path.join(DATA_PATH, "WarAndPeaceEng.txt"), "r", encoding="utf-8") as file:
    text = file.read()
    texts.append(re.sub(RE_EN, "", text.lower()))

test_ru = re.sub(RE_RU, "", TEST.lower())

# Раздел 1. Базовый частотный метод
Реализуем базовый частотный метод по Шерлоку Холмсу:
- подсчитаем частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);  
- возьмем какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруем их посредством случайной перестановки символов;  
- расшифруем их таким частотным методом.

In [22]:
def get_ngram_freq(text: str, n_gram: int=1) -> Dict:
    
    freq = Counter()
    for group_begin in range(len(text) + 1 - n_gram):
        group = text[group_begin : group_begin + n_gram]
        freq[group] += 1

    for key in freq:
        freq[key] /= len(text)

    return freq


corpus_freq = get_ngram_freq(texts[0] + " " + texts[1])

In [23]:
def generate_mapping(freq: Dict) -> Dict:
    original = list(freq.keys())
    replacement = np.random.choice(original, replace=False, size=len(freq))
    mapping = {
        original_char: replacement_char
        for original_char, replacement_char in zip(original, replacement)
    }
    return mapping


def generate_reverse_mapping(corpus_freq: Dict, text_freq: Dict, n_gram: int=1):
    corpus_freq_sorted = sorted(corpus_freq.items(), key=lambda x: x[1], reverse=True)
    text_freq_sorted = sorted(text_freq.items(), key=lambda x: x[1], reverse=True)

    reverse_mapping = {}
    for text_ngram, text_freq in text_freq_sorted:
        filtered_freq = copy(corpus_freq_sorted)

        for j in range(n_gram):
            if text_ngram[j] in reverse_mapping:
                filtered_freq = [
                    (ngram, freq) for ngram, freq in filtered_freq
                    if ngram[j] == reverse_mapping[text_ngram[j]]
                ]

        min_diff = 1
        best_ngram = None
        for ngram, freq in filtered_freq:
            diff = abs(freq - text_freq)
            if diff < min_diff:
                best_ngram = ngram
                min_diff = diff

        for j in range(n_gram):
            if text_ngram[j] not in reverse_mapping:
                reverse_mapping[text_ngram[j]] = best_ngram[j]

    return reverse_mapping


mapping = generate_mapping(corpus_freq)

In [24]:
def apply_mapping(text: str, mapping: Dict) -> str:
    
    new_text = ""
    n_gram = len(list(mapping.keys())[0])
    for i in range(0, len(text) // n_gram * n_gram, n_gram):
        new_text += mapping.get(text[i: i + n_gram])
    return new_text


test_encoded = apply_mapping(test_ru, mapping)

In [25]:
encoded_freq = get_ngram_freq(test_encoded)
reverse_mapping = generate_reverse_mapping(corpus_freq, encoded_freq)
test_decoded = apply_mapping(test_encoded, reverse_mapping)

In [26]:
def accuracy(text, text_decoded):
    correct = sum((char == char_d) for char, char_d in zip(text, text_decoded))
    return correct / len(text)


def describe(text=None, encoded=None, decoded=None, char_per_line=130):
    if text:
        print(
            "Исходный текст:",
            *[text[i:i + char_per_line]for i in range(0, len(text), char_per_line)], 
            "\n",
            end="", 
            sep="\n"
            )
        
    if encoded:
        print(
            "Закодированный текст:",
            *[encoded[i:i + char_per_line] for i in range(0, len(encoded), char_per_line)], 
            "\n",
            end="", 
            sep="\n"
            )
    
    if decoded:
        print(
            "Декодированный текст:",
            *[decoded[i:i + char_per_line] for i in range(0, len(decoded), char_per_line)], 
            "\n",
            end="", 
            sep="\n"
            )
    
    if text and decoded:
        print("Точность:", accuracy(text, decoded), "\n", end="", sep="\n")

In [27]:
describe(test_ru, test_encoded, test_decoded)

Исходный текст:
только окончится один месяц сразу же начинается другой и ни разу еще не бывало так чтобы февраль пришел раньше чем уйдет январь а 
май обогнал бы апрель месяцы идут один за другим и никогда не встречаются но люди рассказывают будто в горной стране богемии была 
девочка которая видела все двенадцать месяцев сразу как же это случилось а вот как в одной маленькой деревушке жила злая и скупая 
женщина с дочкой и падчерицей дочку она любила а падчерица ничем ей не могла угодить что ни сделает падчерица  все не так как ни п
овернется  все не в ту сторону  дочка по целым дням на перине валялась да пряники ела а падчерице с утра до ночи и присесть некогд
а было то воды натаскай то хворосту из лесу привези то белье на речке выполощи то грядки в огороде выполи знала она и зимний холод
 и летний зной и весенний ветер и осенний дождь потомуто может и довелось ей однажды увидеть все двенадцать месяцев разом была зим
а шел январь месяц снегу намело столько что от дверей его приходило

Посимвольная точность небольшая, декодированный текст не понятен.

# Раздел 2. Биграммы
- подсчитаем частоты биграмм (т.е. пар последовательных букв) по корпусам;
- проведем тестирование аналогично п.1, но при помощи биграмм.


In [28]:
corpus_freq_bigram = get_ngram_freq(texts[0] + " " + texts[1], n_gram=2)
encoded_freq_bigram = get_ngram_freq(test_encoded, n_gram=2)

In [29]:
reverse_mapping_bigram = generate_reverse_mapping(corpus_freq_bigram, encoded_freq_bigram, n_gram=2)
test_decoded_bigram = apply_mapping(test_encoded, reverse_mapping_bigram)

In [30]:
describe(test_ru, test_encoded, test_decoded_bigram)

Исходный текст:
только окончится один месяц сразу же начинается другой и ни разу еще не бывало так чтобы февраль пришел раньше чем уйдет январь а 
май обогнал бы апрель месяцы идут один за другим и никогда не встречаются но люди рассказывают будто в горной стране богемии была 
девочка которая видела все двенадцать месяцев сразу как же это случилось а вот как в одной маленькой деревушке жила злая и скупая 
женщина с дочкой и падчерицей дочку она любила а падчерица ничем ей не могла угодить что ни сделает падчерица  все не так как ни п
овернется  все не в ту сторону  дочка по целым дням на перине валялась да пряники ела а падчерице с утра до ночи и присесть некогд
а было то воды натаскай то хворосту из лесу привези то белье на речке выполощи то грядки в огороде выполи знала она и зимний холод
 и летний зной и весенний ветер и осенний дождь потомуто может и довелось ей однажды увидеть все двенадцать месяцев разом была зим
а шел январь месяц снегу намело столько что от дверей его приходило

Результат ухудшился, а вместе с ним и метрика, декодированный текст менее понятен прошлого варианта.

# Раздел 3. MCMC-сэмплирование   
- предложим метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
- реализуем и протестируем его, убедимся, что результаты улучшились.

Будем использовать марковские цепи, представленные текстом разбитым на n-граммы, вероятность перехода между состояниями цепи - частота n-грамм. В качестве оценки считаем произведение всех встречающихся частот n-грамм. Для простоты вычислений используем логарифмическое правдоподобие. Применим алгоритм, основанный на MCMC, на каждой его итерации будем делать несколько циклов и брать в качестве окончательного результата лучший.
- Вычисляем логарифм правдоподобия $p_{current}$;
- Переставляем местами пару символов;
- Восстанавливаем текст с новой перестановкой, вычисляем $p_{proposal}$;
- Новая перестановка принимается с $p_{accept}$, возвращаемся ко второму пункту и повторяем цикл.


In [31]:
def get_ngram_freq_smoothed(text: str, n_gram: int=2):
    vocab = len(set(text)) ** n_gram
    if n_gram > 1:
        text = ["".join(ngram) for ngram in everygrams(text, min_len=n_gram, max_len=n_gram)]

    freq = {k: (v + 1) / (len(text) + vocab) for k, v in Counter(text).items()}
    return freq


def get_text_proba(
        text: str, 
        mapping: Dict, 
        freq: Dict, 
        n_gram: int=2, 
        alphabet: str=ALPHABET_RU
        ):
    text_decoded = apply_mapping(text, mapping)
    log_proba = 0
    for i in range(len(text_decoded) - n_gram):
        ngram = text_decoded[i : i + n_gram]
        ngram_proba = freq.get(ngram, 1 / (len(text) + len(alphabet) ** n_gram))
        log_proba += np.log(ngram_proba)
    return log_proba


def generate_reverse_mapping_mcmc(
    text_encoded: str,
    corpus_freq: Dict,
    alphabet_encoded: str=ALPHABET_RU,
    alphabet_corpus: str=ALPHABET_RU,
    n_iters: int=20000,
    n_trials: int=20,
    n_gram: int=2,
):
    mappings = []
    best_reverse_mapping = None
    best_log_likelihood = - np.inf

    for _ in tqdm(range(n_trials), total=n_trials):
        alphabet_encoded = list(alphabet_encoded)
        alphabet_corpus = list(alphabet_corpus)
        reverse_mapping = {k: v for k, v in zip(alphabet_encoded, alphabet_corpus[: len(alphabet_encoded)])}

        log_proba_current = get_text_proba(text_encoded, reverse_mapping, corpus_freq, n_gram=n_gram)

        for _ in range(n_iters):
            alphabet_proposal = alphabet_corpus[:]
            id_1, id_2 = np.random.choice(len(alphabet_proposal), replace=False, size=2)
            alphabet_proposal[id_1], alphabet_proposal[id_2] = alphabet_proposal[id_2], alphabet_proposal[id_1]

            reverse_mapping_proposal = {k: v for k, v in zip(alphabet_encoded, alphabet_proposal[: len(alphabet_encoded)])}
            log_proba_proposal = get_text_proba(text_encoded, reverse_mapping_proposal, corpus_freq, n_gram=n_gram)

            p = np.exp(log_proba_proposal - log_proba_current)

            if p > np.random.rand():
                alphabet_corpus = alphabet_proposal
                log_proba_current = log_proba_proposal
                reverse_mapping = reverse_mapping_proposal

        if log_proba_current > best_log_likelihood:
            best_log_likelihood = log_proba_current
            best_reverse_mapping = reverse_mapping

        mappings.append(reverse_mapping)

    print(f"Log likelihood: {best_log_likelihood}")

    return best_reverse_mapping

In [32]:
corpus_freq = get_ngram_freq_smoothed(texts[0] + " " + texts[1], n_gram=2)
reverse_mapping_mcmc = generate_reverse_mapping_mcmc(test_encoded, corpus_freq)
test_decoded_mcmc = apply_mapping(test_encoded, reverse_mapping_mcmc)

100%|██████████| 20/20 [07:19<00:00, 21.99s/it]

Log likelihood: -6249.856327986885


In [33]:
describe(test_ru, test_encoded, test_decoded_mcmc)

Исходный текст:
только окончится один месяц сразу же начинается другой и ни разу еще не бывало так чтобы февраль пришел раньше чем уйдет январь а 
май обогнал бы апрель месяцы идут один за другим и никогда не встречаются но люди рассказывают будто в горной стране богемии была 
девочка которая видела все двенадцать месяцев сразу как же это случилось а вот как в одной маленькой деревушке жила злая и скупая 
женщина с дочкой и падчерицей дочку она любила а падчерица ничем ей не могла угодить что ни сделает падчерица  все не так как ни п
овернется  все не в ту сторону  дочка по целым дням на перине валялась да пряники ела а падчерице с утра до ночи и присесть некогд
а было то воды натаскай то хворосту из лесу привези то белье на речке выполощи то грядки в огороде выполи знала она и зимний холод
 и летний зной и весенний ветер и осенний дождь потомуто может и довелось ей однажды увидеть все двенадцать месяцев разом была зим
а шел январь месяц снегу намело столько что от дверей его приходило

Результат приятно удивляет, точное полное декодирование.

# Раздел 4. Расшифровка сообщения  

In [34]:
msg = "დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ"

In [35]:
corpus_freq = get_ngram_freq(texts[0] + " " + texts[1])
msg_freqs = get_ngram_freq(msg)

corpus_freq_sorted = sorted(corpus_freq.items(), key=lambda x: x[1], reverse=True)
msg_freq_sorted = sorted(msg_freqs.items(), key=lambda x: x[1], reverse=True)

alphabet_corpus = "".join([char for char, _ in corpus_freq_sorted])
alphabet_encoded = "".join([char for char, _ in msg_freq_sorted])

corpus_freq = get_ngram_freq_smoothed(texts[0] + " " + texts[1], n_gram=2)
reverse_mapping_msg = generate_reverse_mapping_mcmc(
    msg,
    alphabet_encoded=alphabet_encoded,
    alphabet_corpus=alphabet_corpus,
    corpus_freq=corpus_freq
)
msg_decoded = apply_mapping(msg, reverse_mapping_msg)

100%|██████████| 20/20 [01:22<00:00,  4.12s/it]

Log likelihood: -1232.8845821292682


In [36]:
present_msg = "если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаю"
describe(present_msg, msg, msg_decoded)

Исходный текст:
если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правиль
но и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаю

Закодированный текст:
დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵი
სႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ

Декодированный текст:
если вы вимите нордальный или почти нордальный текст у этого соожшения который легко прочитать скорее всего вы все смелали правиль
но и получите даксидальный жалл за послемнее четвертое замание курса ботя конечно я ничего не ожешаю

Точность:
0.9391304347826087



Заметна путаница с буквами "м" / "д", "б" / "ж", "щ" и "х", но смысл сообщения улавливается.

# Раздел 5. Триграммы
А что если от биграмм перейти к триграммам (тройкам букв) или даже больше? Улучшатся ли результаты? Когда улучшатся, а когда нет? Чтобы ответить на этот вопрос эмпирически, уже может понадобиться погенерировать много тестовых перестановок и последить за метриками, глазами может быть и не видно.

In [37]:
corpus_freq = get_ngram_freq(texts[0] + " " + texts[1])
msg_freqs = get_ngram_freq(msg)

corpus_freq_sorted = sorted(corpus_freq.items(), key=lambda x: x[1], reverse=True)
msg_freq_sorted = sorted(msg_freqs.items(), key=lambda x: x[1], reverse=True)

alphabet_corpus = "".join([char for char, _ in corpus_freq_sorted])
alphabet_encoded = "".join([char for char, _ in msg_freq_sorted])

corpus_freq_trigram = get_ngram_freq_smoothed(texts[0] + " " + texts[1], n_gram=3)
reverse_mapping_msg_trigram = generate_reverse_mapping_mcmc(
    msg,
    alphabet_encoded=alphabet_encoded,
    alphabet_corpus=alphabet_corpus,
    corpus_freq=corpus_freq_trigram,
    n_iters=10000,
    n_trials=10,
    n_gram=3
)
msg_decoded_trigram = apply_mapping(msg, reverse_mapping_msg_trigram)

100%|██████████| 30/30 [03:16<00:00,  6.55s/it]

Log likelihood: -1851.4135910201478


In [38]:
describe(present_msg, msg, msg_decoded_trigram)

Исходный текст:
если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правиль
но и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаю

Закодированный текст:
დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵი
სႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ

Декодированный текст:
если жт жидине вормалывть или почни вормалывть нексн у эного сообщевия конорть легко прочинаны скорее жсего жт жсе сделали пражилы
во и получине максималывть балл за последвее ченжерное задавие курса хоня ковечво я вичего ве обещац

Точность:
0.7913043478260869



# Раздел 6. Применение модели
Данная модель может быть применима для работы с последовательностями ДНК, например, для сопоставления определенных генетических последовательностей, которые могут быть расшифрованы в соответствии с определенными признаками организма или для оценки взаимодействий генов между собой.